In [ ]:
# importing necessary libraries and functions
import numpy as np
from flask import Flask, request, jsonify, render_template
import pickle
import joblib
from sklearn.linear_model import LogisticRegression
# Importing necessary libraries
import numpy as np
import pandas as pd 
from pandas import Series, DataFrame
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
import pickle
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 

# Import CountVectorizer from feature_extraction.text.
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV


app = Flask(__name__) #Initialize the flask App
modelnlp = pickle.load(open('modelnlp.pkl', 'rb')) # loading the trained model

@app.route('/') # Homepage
def home():
    return render_template('index.html')

@app.route('/predict',methods=['GET','POST'])
def predict():
    '''
    For rendering results on HTML GUI
    '''
    
    train_colorectal = pd.read_csv("train_colorectal.csv")
    
    y = train_colorectal['Class']
    
    X = train_colorectal['Text']
    
    x_train, x_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    stratify=y,
                                                    random_state=42
                                                    )
    class LemmaTokenizer:
        def __init__(self):
            self.wnl = WordNetLemmatizer()
        def __call__(self, doc):
            return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]
    
    cvec = CountVectorizer(analyzer='word', tokenizer=LemmaTokenizer(), ngram_range=(1, 1))
    
    cvec.fit(x_train)
    
    x_train = cvec.transform(x_train)
    
    parameters = {'C': [0.001, 0.01, 0.1, 1, 10],
              'class_weight': [None, 'balanced'],
              'penalty': ['l1', 'l2']}
    
    lr = LogisticRegression(solver = 'liblinear', 
                        max_iter = 1000,
                        random_state = 42)
    modelnlp = GridSearchCV(estimator = lr,                                    # Specify the model we want to GridSearch.
                          param_grid = parameters,                           # Specify the grid of parameters we want to search.
                          scoring = 'accuracy',                                # Specify recall as the metric to optimize 
                          cv = 5).fit(x_train, y_train)
    
    modelnlp.fit(x_train,y_train)
    modelnlp.score(x_test,y_test)
#Alternative Usage of Saved Model
# joblib.dump(clf, 'NB_spam_model.pkl')
# NB_spam_model = open('NB_spam_model.pkl','rb')
# clf = joblib.load(NB_spam_model)

    if request.method == 'POST':
        message = request.form['message']
    data = [message]
    vect = cv.transform(data).toarray()
    my_prediction = modelnlp.predict(vect)
    return render_template('result.html',prediction = my_prediction)
    
    # retrieving values from form
    #init_features = [str(x) for x in request.form.values()]
    #final_features = [np.array(init_features)]

    #prediction = model.predict(final_features) # making prediction


    return render_template('index.html', prediction_text='Predicted Class: {}'.format(prediction)) # rendering the predicted result

if __name__ == "__main__":
    app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Jun/2021 19:42:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2021 19:42:42] "GET /static/css/style.css HTTP/1.1" 404 -
C:\Users\Tresha\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
127.0.0.1 - - [03/Jun/2021 19:46:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2021 19:46:57] "GET /static/css/style.css HTTP/1.1" 404 -
C:\Users\Tresha\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
127.0.0.1 - - [03/Jun/2021 19:47:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2021 19:47:53] "GET /static/css/style.css HTTP/1.1" 404 -
127.0.0.1 - - [03/Jun/2021 19:48:48] "GE